# Databricks Autoloader Ingest

Placeholder notebook. Use Databricks Autoloader to ingest employment contracts from Blob Storage into Delta Bronze.

In [ ]:
from pyspark.sql.functions import input_file_name, current_timestamp, lit
from pyspark.sql.types import StructType, StructField, BinaryType, StringType

raw_path = "abfss://contracts@ragstorage4122025.dfs.core.windows.net/"
bronze_path = "abfss://bronze@ragstorage4122025.dfs.core.windows.net/"
checkpoint = "dbfs:/checkpoints/contracts/bronze/"

df = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "binary")
        .option("cloudFiles.schemaLocation", checkpoint + "/schema")
        .load(raw_path)
)

df = df.withColumn("filename", input_file_name()) \
       .withColumn("ingest_timestamp", current_timestamp())

(
    df.writeStream
      .format("delta")
      .option("checkpointLocation", checkpoint)
      .trigger(availableNow=True)      # or continuous=true
      .outputMode("append")
      .start(bronze_path)
)
